# Importiamo le librerie necessarie

In [3]:
# data manipulation
import pandas as pd

# numpy arrays
import numpy as np

# data visualization
import seaborn as sns

import matplotlib.pyplot as plt

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

sns.set()

# NLP
import string

from wordcloud import WordCloud

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from Code.NLTKVectorizer import NLTKVectorizer

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.naive_bayes import MultinomialNB  # Naive Bayes
from sklearn.svm import LinearSVC  # SVM
from sklearn.ensemble import RandomForestClassifier  # Random Forest

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

# Model explainability
from lime.lime_text import LimeTextExplainer

# other
from pprint import pprint
from time import time
import logging
from functools import partial
import joblib

#nltk

nltk.download('punkt')

nltk.download('stopwords')

nltk.download('wordnet')

en_stop_words = list(set(stopwords.words("english")))

# aggiungo a en_stop_words le parole dal file da noi creato
with open('stopwordsPersonali', 'r') as file:
    for line in file:
        en_stop_words.append(line.strip())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giaco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giaco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giaco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 assegniamo i dataset di training e test alle variabili di train e test

## Setup dati iniziali

In [4]:
dfTrain = pd.read_csv('Dataset/archive/train.csv')
dfTest = pd.read_csv('Dataset/archive/test.csv')

In [5]:
X_train = dfTrain['question_content']
y_train = dfTrain['topic']
X_test = dfTest['question_content']
y_test = dfTest['topic']

## Generatore della color palette

Questa funzione viene utilizata per creare una *palette* di `n` colori di `palette_name` colori.

In [2]:
def get_n_color_palette(palette_name, n_colors, as_hex=False):
    palette = sns.color_palette(palette=palette_name, n_colors=n_colors)
    if as_hex:
        palette = palette.as_hex()
    palette.reverse()
    return palette

## Plotly export chart 

Questa funzione è utilizzata per esportare l' HTML di plotly `fig_obj`, e salvarlo in: `assets/file_name.html`

In [3]:
def save_fig_as_div(fig_obj, file_name):
    with open(f"{file_name}", "w") as fig_file:
        fig_div_string = plotly.offline.plot(
            figure_or_data=fig_obj, output_type="div", include_plotlyjs="cdn"
        )
        fig_file.write(fig_div_string)

La funzione genera il report di classificazione per le previsioni del modello

## Funzione per il confusion matrix

Questa funzione genera una confusion map per le previsioni del modello

In [6]:
def get_confusion_matrix(y_true, y_pred, labels):

    # calcola la confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels)
    
    
    sum = 0
    diag = 0
    
    for i in range(len(conf_matrix)):
        for j in range(len(conf_matrix[i])):
            sum += conf_matrix[i][j]
            if i == j:
                diag += conf_matrix[i][j]
            
    
    
    print(f"Diagonale: {diag}, Totale: {sum}")
    print(f"Accuracy: {(diag/sum)*100}%")
    
    
    
    conf_matrix = np.flipud(conf_matrix)

    # crea una heatmap annotata della matrice di confusione
    fig = ff.create_annotated_heatmap(
        conf_matrix, x=labels.tolist(), y=labels.tolist()[::-1]
    )
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        title_text="<i><b>Confusion matrix</b></i>",
        xaxis_title="Predicted category",
        yaxis_title="Real category",
        plot_bgcolor="rgba(0, 0, 0, 0)",
        paper_bgcolor="rgba(0, 0, 0, 0)",
        font={
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    )
    fig.update_xaxes(tickangle=-45)
    fig["data"][0]["showscale"] = True

    return fig

# Funzione per il classification report

In [4]:
def get_classification_report(y_true, y_pred, target_names):

    # calcola il report di classificazione e lo converte in un DataFrame
    
    clf_report = classification_report(
        y_true=y_true, y_pred=y_pred, target_names=target_names, output_dict=True
    )
    clf_report_df = pd.DataFrame(data=clf_report)
    clf_report_df = clf_report_df.T
    clf_report_df.drop(columns=["support"], inplace=True)

    measures = clf_report_df.columns.tolist()
    classes = clf_report_df.index.tolist()

    # crea un heatmap annotato di plotly e aggiorna lo stile
    
    fig = ff.create_annotated_heatmap(clf_report_df.values, x=measures, y=classes)
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        title_text="<i><b>Classification report</b></i>",
        xaxis_title="Measures",
        yaxis_title="Class",
        plot_bgcolor="rgba(0, 0, 0, 0)",
        paper_bgcolor="rgba(0, 0, 0, 0)",
        font={
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    )
    fig.update_xaxes(tickangle=-45)
    fig["data"][0]["showscale"] = True

    return fig

## Data Statistics

Calcola per ogni topic nel dataset il numero di domande per topic

In [6]:
categories_statistics_df = (
    dfTrain.groupby(by="topic")["id"]
    .agg(
        [
            ("count", lambda x: x.size),
        ]
    )
    .reset_index()
    .sort_values(by="count", ascending=False)
)

NameError: name 'dfTrain' is not defined

Calcola per ogni topic nel dataset la lunghezza media delle domande per topic

In [32]:
categories_statistics_df_questions = (
    dfTrain.groupby(by="topic")["question_content"]
    .agg(
        [
            ("mean", lambda x: x.str.len().mean()),
            ("max", lambda x: x.str.len().max()),
            ("min", lambda x: x.str.len().min()),
        ]
    )
    .reset_index()
)

## Grafico domande per topic

Usa un grafico a torta per mostrare le percentuali di domande per ogni topic:

In [ ]:
blue_palette = get_n_color_palette("Blues", 20, True)

fig = px.pie(
    data_frame=categories_statistics_df,
    names="topic",
    values="count",
    color_discrete_sequence=blue_palette,
    title="Percentuale di domande per topic",
    width=800,
    height=500,
)

fig.update_layout(
    {
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
        "paper_bgcolor": "rgba(0, 0, 0, 0)",
        "font": {
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    }
)

fig.show()


Possiamo vedere che il dataset è *bilanciato*

In [18]:
# salvo il grafico in un file html
save_fig_as_div(fig, file_name='charts/categories-percentages-pie-chart.html')

## Grafico lunghezza media delle domande per topic:

Usa un diagramma a barre per mostrare la lunghezza media delle domande per ogni topic:

In [ ]:
chart_labels = {"mean": "Lunghezza delle domande", "Topic": "Topic type"}

fig = px.bar(
    data_frame=categories_statistics_df_questions.sort_values(by="mean"),
    x="topic",
    y="mean",
    color="mean",
    labels=chart_labels,
    title="Lunghezza media delle domande per topic",
    width=800,
    height=500,
)

fig.update_layout(
    {
        "plot_bgcolor": "rgba(0, 0, 0, 0)",
        "paper_bgcolor": "rgba(0, 0, 0, 0)",
        "font": {
            "family": "Courier New, monospace",
            "size": 14,
            # 'color': "#eaeaea"
        },
    }
)

# rotate x-axis ticks
fig.update_xaxes(tickangle=-45)

fig.show()

Notiamo che la lunghezza delle domande è ben distrubuita per tutti i topic, tranne per il topic 0 (Society & Culture), 8 (Family & Relationships) e 9 (Politics & Government), che hanno una lunghezza media delle domande leggermente più lunga rispetto agli altri topic.

In [21]:
save_fig_as_div(fig, file_name="charts/average-article-length-bar-chart.html")

# Word Cloud

In [23]:
categories_text_df = dfTrain.groupby(by="topic").agg({"question_content": " ".join}).reset_index()

In [24]:
def plot_word_cloud(category_name, category_text):
    plt.subplots(figsize=(8, 8))
    wc = WordCloud(
        background_color="white", stopwords=en_stop_words, width=1000, height=600
    )
    wc.generate(category_text)
    plt.title(label=category_name)
    plt.axis("off")
    plt.imshow(wc, interpolation="bilinear")
    plt.show()

La seguente word-cloud ci aiuterà a dare uno sguardo ai dati e al suo contenuto.

Per ogni word cloud, le parole con una frequenza maggiore hanno una dimensione maggiore.

Questo ci aiuterà a capire quali sono le parole più frequenti in ogni topic.

In [ ]:
for idx, row in categories_text_df.iterrows():
    plot_word_cloud(row["topic"], row["question_content"])

Notiamo che le parole più comunemente usate, che però non rispecchiano il topic sono: 
- think
- would
- get
- want

Quindi sono state aggiunte alla lista di stopwords.

# Text Vectorization

In questo passaggio, costruiremo un **text vectorization** transformer che verrà utilizzato per convertire le domande grezze in funzionalità appropriate, preparate per essere inserite negli algoritmi di machine learning.

Definiamo un *custom vectorizer* chiamato `NLTKVectorizer`. Questo vectorizer eredita il `TfidfVectorizer` e sovrascrive il metodo `build_analyzer`. Il vectorizer risultante avrà gli stessi parametri del `TfidfVectorizer` ma analizzerà i documenti in modo diverso, principalmente utilizzerà il tokenzier `NLTK`, lemmatizer.

## Vectorizer e tipi di modelli

Qui di seguito viene definito il vectorizer e i modelli che verranno presi in considerazione

In [61]:
# text vectorizer
vectorizer = NLTKVectorizer(
    stop_words=en_stop_words, max_df=0.5, min_df=10, max_features=10000
)

# Logistic Regression classifier
lr_clf = LogisticRegression(C=1.0, solver="newton-cg", multi_class="multinomial", n_jobs=1, verbose=2)

# Naive Bayes classifier
nb_clf = MultinomialNB(alpha=0.01)

# SVM classifier
svm_clf = LinearSVC(C=1.0, verbose=2)

# Random Forest classifier
random_forest_clf = RandomForestClassifier(
    n_estimators=100, criterion="gini", max_depth=50, random_state=0, n_jobs=1, verbose=2)



In [62]:
# create pipeline object
pipeline = Pipeline([("vect", vectorizer), ("clf", lr_clf)])
#pipeline = Pipeline([("vect", vectorizer), ("clf", svm_clf)])
#pipeline = Pipeline([("vect", vectorizer), ("clf", random_forest_clf)])


# Creazione della pipeline

Creo la pipeline con il modello migliore e il vectorizer a cui passo le stopwords

In [ ]:
%%time
pipeline.fit(X_train, y_train)

In [ ]:
%%time
y_pred = pipeline.predict(X_test)
#y_pred = model.predict(X_test)


## Classification report:

In [ ]:
target_names = [0, 1, 2, 3 ,4 ,5 ,6, 7, 8 ,9]
target_names=list(map(str,target_names))
classes = target_names
fig = get_classification_report(
    y_true=y_test, y_pred=y_pred, target_names=classes
)
fig.show()

In [37]:
save_fig_as_div(fig_obj=fig, file_name="charts/classification-report_svc.html")

## Confusion matrix con calcolo diagonale e totale:

In [ ]:
fig = get_confusion_matrix(y_true=y_test, y_pred=y_pred, labels=pipeline.classes_)
fig.show()

In [60]:
save_fig_as_div(fig_obj=fig, file_name="charts/confusion-matrix_lr.html")

## Grid Search

Tramite la grid search possiamo trovare i migliori parametri per il migliore modello da utilizzare

In [ ]:
parameters = {
    # vectorizer hyper-parameters
    #'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vect__ngram_range': [(1, 1)],
    #'vect__max_df': [0.4, 0.5, 0.6],
    'vect__max_df': [0.5],
    #'vect__min_df': [10, 50, 100],
    'vect__min_df': [10],
    #'vect__max_features': [5000, 10000],
    'vect__max_features': [10000],
    # classifiers
    'clf': [lr_clf, nb_clf, svm_clf, random_forest_clf]
}
grid_search = GridSearchCV(pipeline, parameters, n_jobs=12)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

## Model Dump

In [1]:
from joblib import dump, load

In [ ]:
dump(pipeline, 'dump/model_lr.joblib')

In [2]:
pipeline = load('dump/model_lr.joblib')

## Carichiamo il modello e utilizziamolo per fare delle previsioni

In [ ]:
text = input('Enter your question: ')

predictProba = pipeline.predict_proba([text])

array = ["Society & culture", "Science & Mathematics", "Health", "Education & Reference", "Computers & Internet", "Sports", "Business & Finance", "Entertainment & Music", "Family & Relationships", "Politics & Government"]
stat = predictProba[0][pipeline.predict([text])[0]] * 100

print("\"" + text + "\"" + "\n" " is about: " + "\"" + array[pipeline.predict([text])[0]]+ "\"" +"\n"+" with a probability of " +"%.3f" %stat + "%" "\n")

for i in range(10):
    print("\t\t\t"+ array[i] + " : " + "%.3f" % (predictProba[0][i] * 100) + "%")